In [1]:
from google.colab import drive
drive.mount('/content/drive')
!tar -xzf "/content/drive/MyDrive/Blood/noaug_coco.tar.gz"
!git clone https://github.com/ultralytics/yolov5.git
!cd yolov5 && pip install -r requirements.txt
!pip install torch torchvision

Mounted at /content/drive
Cloning into 'yolov5'...
remote: Enumerating objects: 17413, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 17413 (delta 64), reused 22 (delta 22), pack-reused 17327 (from 4)
Receiving objects: 100% (17413/17413), 16.31 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (11932/11932), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB

In [3]:
import os
import json
from pathlib import Path

def convert_coco_to_yolo(json_file, image_dir, output_label_dir):
    """
    Convert COCO JSON annotations to YOLO format.
    :param json_file: Path to JSON file (e.g., train.json)
    :param image_dir: Directory with images
    :param output_label_dir: Directory to save YOLO .txt files
    """
    # Create output directory
    Path(output_label_dir).mkdir(parents=True, exist_ok=True)

    # Load JSON
    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    # Create image ID to filename mapping
    image_id_to_filename = {img['id']: img for img in coco_data['images']}

    # Initialize label files (clear existing content)
    for img in coco_data['images']:
        label_file = os.path.join(output_label_dir, Path(img['file_name']).stem + '.txt')
        open(label_file, 'w').close()  # Create empty file

    # Process annotations
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        if image_id not in image_id_to_filename:
            print(f"Warning: Image ID {image_id} not found in images")
            continue

        img_info = image_id_to_filename[image_id]
        img_width, img_height = img_info['width'], img_info['height']
        file_name = img_info['file_name']

        # Get bounding box [x_min, y_min, width, height]
        x_min, y_min, width, height = ann['bbox']

        # Convert to YOLO format (normalized center_x, center_y, width, height)
        center_x = (x_min + width / 2) / img_width
        center_y = (y_min + height / 2) / img_height
        width = width / img_width
        height = height / img_height

        # Clip values to [0, 1]
        center_x = min(max(center_x, 0), 1)
        center_y = min(max(center_y, 0), 1)
        width = min(max(width, 0), 1)
        height = min(max(height, 0), 1)

        # YOLO class_id is 0-based (COCO category_id is 1-based)
        class_id = ann['category_id'] - 1

        # Write to label file
        label_file = os.path.join(output_label_dir, Path(file_name).stem + '.txt')
        with open(label_file, 'a') as f:
            f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")

    print(f"Converted {json_file} to YOLO format")

# Define paths
dataset_root = '/content/noaug_coco'
splits = ['train', 'val', 'test']

# Convert annotations for each split
for split in splits:
    json_file = os.path.join(dataset_root, 'annotations', f'{split}.json')
    image_dir = os.path.join(dataset_root, split)
    output_label_dir = os.path.join(dataset_root, f'labels/{split}')
    convert_coco_to_yolo(json_file, image_dir, output_label_dir)

print("Conversion complete!")

Converted /content/noaug_coco/annotations/train.json to YOLO format
Converted /content/noaug_coco/annotations/val.json to YOLO format
Converted /content/noaug_coco/annotations/test.json to YOLO format
Conversion complete!


In [4]:
import shutil
from pathlib import Path

dataset_root = '/content/noaug_coco'
for split in ['train', 'val', 'test']:
    src_dir = Path(dataset_root) / split
    dst_dir = Path(dataset_root) / 'images' / split
    dst_dir.mkdir(parents=True, exist_ok=True)
    for img in src_dir.glob('*.jpg'):
        shutil.copy(img, dst_dir / img.name)

for split in ['train', 'val', 'test']:
    src_dir = Path(dataset_root) / split
    if src_dir.exists():
        shutil.rmtree(src_dir)
        print(f"Deleted {src_dir}")

Deleted /content/noaug_coco/train
Deleted /content/noaug_coco/val
Deleted /content/noaug_coco/test


In [5]:
class_names = ['ba', 'eo', 'erb', 'ig', 'lym', 'mono', 'neut', 'platelet']
num_classes = len(class_names)
yaml_content = f"""
train: /content/noaug_coco/images/train
val: /content/noaug_coco/images/val
test: /content/noaug_coco/images/test

nc: {num_classes}
names: {class_names}
"""
with open('/content/noaug_coco/data.yaml', 'w') as f:
    f.write(yaml_content)
print(f"Created data.yaml with {num_classes} classes: {class_names}")

Created data.yaml with 8 classes: ['ba', 'eo', 'erb', 'ig', 'lym', 'mono', 'neut', 'platelet']


In [6]:
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 50 --data /content/noaug_coco/data.yaml --weights yolov5s.pt --cache

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      46/49      4.67G    0.01415   0.005688   0.002144         37        640:  54% 387/717 [02:14<02:12,  2.48it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      46/49      4.67G    0.01415   0.005693   0.002146         44        640:  54% 388/717 [02:15<02:41,  2.04it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      46/49      4.67G    0.01415   0.005689   0.002142         32        640:  54% 389/717 [02:15<02:21,  2.32it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(

In [7]:
!cp -r /content/yolov5/runs/train/exp /content/drive/MyDrive/Blood/yolov5_results/

!cd yolov5 && python val.py --img 640 --data /content/noaug_coco/data.yaml --weights /content/yolov5/runs/train/exp/weights/best.pt


!cd yolov5 && python detect.py --img 640 --conf 0.25 --source /content/noaug_coco//images/test --weights /content/yolov5/runs/train/exp/weights/best.pt
!cp -r /content/yolov5/runs/detect/exp /content/drive/MyDrive/yolov5_results/detect/

val: data=/content/noaug_coco/data.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/noaug_coco/labels/val.cache... 2100 images, 0 backgrounds, 0 corrupt: 100% 2100/2100 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 66/66 [00:31<00:00,  2.11it/s]
                   all       2100       4461      0.976      0.946      0.992      0.918
                    ba       2100        302      0.993      0.987      0.995      0.935
     

In [10]:
!cp -r /content/yolov5/runs/detect/exp /content/drive/MyDrive/Blood/yolov5_results/detect